In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score ipywidgets

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
from pyarrow.dataset import dataset
from sympy.codegen.ast import continue_
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig, TFOpenAIGPTDoubleHeadsModel
)
from tqdm import tqdm
from trl import SFTTrainer
import json
import torch
import time
import random
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login
from datasets import load_dataset

interpreter_login()

In [ ]:
import os
# disable Weights and Biases # TODO evaluate using WB
os.environ['WANDB_DISABLED']="true"
os.environ['PYTORCH_CUDA_ALLOC_CONF']="expandable_segments:True"

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
from time import sleep

progress_bar = IntProgress(min=0, max=100)
display(progress_bar)

for i in range(100):
    progress_bar.value = i
    sleep(0.1)

In [ ]:
# Carrega o dataset e divide ele em treino, validação e teste (TODO montar o dataset já desta forma)
dataset = load_dataset('json', data_files='../data_construction/dataset_protein_articles_trunc.jsonl', split='train')

# Definir proporções simplificadas (4:1:1) 
train_ratio = 4 / (4 + 1 + 1)
test_ratio = 1 / (4 + 1 + 1)
validation_ratio = 1 / (4 + 1 + 1)

# Primeiro, dividir o dataset entre treino e (teste + validação)
train_val_test = dataset.train_test_split(test_size=(1 - train_ratio))

# Dividir a parte restante entre teste e validação
test_val = train_val_test['test'].train_test_split(test_size=validation_ratio / (test_ratio + validation_ratio))

# Obter os datasets de treino, teste e validação
train_dataset = train_val_test['train']
test_dataset = test_val['train']
validation_dataset = test_val['test']

# Exibir o tamanho de cada split
print(f"Tamanho do treino: {len(train_dataset)}")
print(f"Tamanho do teste: {len(test_dataset)}")
print(f"Tamanho da validação: {len(validation_dataset)}")


In [ ]:
compute_dtype = getattr(torch, "float16")
model_name='meta-llama/Llama-3.2-3B'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:

device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      token=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['instruction', 'input', 'output'],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    blurb = f"\nBelow is an instruction that describes a task. Write a response that appropriately completes the request."
    instruction = f"### Instruct: {sample['instruction']}"
    response = f"### Output: {sample['output']}"
    end = f"### End"

    parts = [part for part in [blurb, instruction, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [ ]:
from transformers import set_seed
seed = 42 # TODO check this
set_seed(seed)

## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, train_dataset)
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, validation_dataset)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=32, 
    lora_alpha=32, 
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05, 
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [ ]:
# 2 - Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA
original_model = prepare_model_for_kbit_training(original_model)

In [ ]:
output_dir = f'./peft-protein-model-{str(int(time.time()))}'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    eval_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = True,
    group_by_length=True,
    disable_tqdm=False
)

peft_model.config.use_cache = False
peft_model.requires_grad = True

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
peft_trainer.train()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      token=True)

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/mnt/TCC/protein-chat-flask/fine_tuning/peft-dialogue-summary-training-1728237910/checkpoint-1000",torch_dtype=torch.float16,is_trainable=False)

In [ ]:
def gen(model,p, maxlen=100, sample=True):
    toks = tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.01,num_beams=1,top_p=0.95,)
    return tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 6
instruction = test_dataset[index]['instruction']

prompt = f"Instruct: {instruction}\nOutput:\n"

peft_model_res = gen(ft_model,prompt,100,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
#print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')